# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235159976494                   -0.50    7.0
  2   -7.250379170546       -1.82       -1.40    1.0
  3   -7.251268420386       -3.05       -2.08    4.0
  4   -7.251112312066   +   -3.81       -1.99    4.0
  5   -7.251333759956       -3.65       -2.80    2.0
  6   -7.251338454708       -5.33       -3.30    2.0
  7   -7.251338788426       -6.48       -4.14    2.0
  8   -7.251338794613       -8.21       -4.25    4.0
  9   -7.251338798513       -8.41       -4.86    3.0
 10   -7.251338798670       -9.80       -5.35    2.0
 11   -7.251338798702      -10.50       -5.83    3.0
 12   -7.251338798704      -11.64       -6.22    2.0
 13   -7.251338798705      -12.56       -7.09    3.0
 14   -7.251338798705      -14.27       -7.28    3.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.047727105748504944
[ Info: Arnoldi iteration step 2: normres = 0.6113633779584655
[ Info: Arnoldi iteration step 3: normres = 0.7786102012602935
[ Info: Arnoldi iteration step 4: normres = 0.4652802603354386
[ Info: Arnoldi iteration step 5: normres = 0.34898049285073834
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.56e-02, 7.54e-02, 2.16e-01, 2.57e-01, 1.42e-02)
[ Info: Arnoldi iteration step 6: normres = 0.35696377163502513
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.78e-02, 1.66e-01, 2.96e-01, 6.93e-02, 7.33e-02)
[ Info: Arnoldi iteration step 7: normres = 0.051803035984344666
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.46e-04, 9.83e-03, 1.01e-02, 1.22e-02, 4.19e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10126261156068006
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.9